## 1 - Introdução

A astronomia observacional é uma área bastante ampla, abrangendo grandes áreas como o estudo do sistema solar, astrofísica estelar, estrutura de galáxias, astrofísica extragaláctica, cosmologia, entre outros. A cosmologia é a área responsável por estudar a origem, evolução, composição e estrutura do universo. Dentro da cosmologia existe o estudo que visa entender a evolução das galáxias e as suas distâncias em relação a Terra, esse estudo utiliza das técnicas de fotometria e espectroscopia de galáxias. Tudo no universo está em movimento, e de maneira geral, em relação a terra, os objetos podem estar se aproximando (blueshift), aparentemente parado ou se afastando (redshift). Esse fenômeno foi primeiramente observado por volta de 1929 por Edwin Hubble utilizando a espectroscopia de galáxias, foi observado que a maioria dessas galáxias possuiam um desvio para o vermelho (redshift), esse estudo originou, mais tarde, a lei de Hubble. Atualmente, o universo pode ser estudado de maneira singular, onde se escolhem objetos para os telescópios observarem e extraírem dados mais específicos, ou em conjunto, onde se varre o céu para extrair a maior quantidade de informações de maneira mais geral sobre um grupo de objetos. 

Dentro do estudo de redshift de galáxias a área de tecnologia da informação está extremamente envolvida em todas as etapas do processo, desde a captura dos dados, na construção das imagens que vemos do universo e nas análises científicas, entre outros. Quando juntamos o cálculo de redshift de galáxias, em maior quantidade, com algoritmos, existem duas técnicas principais para processar esses dados: template fitting e machine learning. Para machine learning temos algoritmos que envolvem treinar uma inteligência artificial (IA) para aprender a calcular redshifts, para isso precisa existir uma amostra com resultados já conhecidos para alimentar a IA, e para o template fitting é utilizado algoritmos que fazem comparação  com dados espectrográficos (templates) existentes de galáxias para determinar resultados dos dados fotométricos de entrada.




## 2 - Dados
Descrição dos dados, resumo do QA (reproduzir Figs. 2, 3 e 4 do paper)

## 3 - Metodologia

Descrever BPZ e métricas

## 4 - Resultados

Mostrar plots e métricas, comparação com os resultados do artigo

## 5 - Conclusões

Lições aprendidas.